# Exploring Gender Biases in Word2vec <br> 

Write a short intro here 


In [20]:
#Imports
import torch
from torch import nn, optim, sigmoid, softmax
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
import string
import numpy as np

from gutenberg_data import get_urls, read_data_from_urls


## 1- Define Word2Vec Model 

In [ ]:
# NOTES:
# inspiration: https://github.com/OlgaChernytska/word2vec-pytorch
# We are using the skip gram method because it works better on smaller data sets (which we are limited to), source -  https://arxiv.org/pdf/1301.3781.pdf 
# Skim-gram predicts a context word from a given middle word in the sentance.
# Use adam optimizer (better than SGD) - source: _______
# Applying regularization to our embedding layer: clamping each weiaght vector of each neuron to |w| < 1 ( via max_norm ==1) - prevent exploding gradients
  #the norm restrictions also helps push cosine similarity between words (which we will use to analyze bias)



**Create Tokens** <br>

Justifications:

- punctuation: remove - source : ______________
- lowercase words - because we are only looking at biasis between male/female

In [6]:
def create_tokens(data, min_freq):
    """
    create_vocabulary finds the number of occurences of each word, creates vocuabulary with words with > N_freq, and lebels them with ids

    :param data: list of strings
    :return: a list of tokens 
    """ 
    tokens = []
    for sentance in data:
      #split into tokens, convert to lowercase 
      sentance = sentance.translate(str.maketrans('', '', string.punctuation)) #remove punctuations
      sentance = sentance.translate(str.maketrans('', '', string.digits)) #remove digits
      tokenizer = get_tokenizer("basic_english", language="en") #remove unessasary characters, splits into spaces
      tokens.append(tokenizer(sentance))
  
    return tokens
  

**Define word 2 vec model** <br>

- choosing the top N most frequent words - why? so its easier to train - and not wasting time to train words that appear once in a text sequence. 

In [15]:
# get test data so far
test_urls = get_urls('test')
test_data = read_data_from_urls(test_urls)
test_data=str(test_data[0]).split('.')

<class 'bytes'>


In [34]:
from gensim.models import Word2Vec
min_freq = 50

sentences = create_tokens(test_data, min_freq)

model = Word2Vec(min_count=1)
model.build_vocab(sentences)  # prepare the model vocabulary
model.train(sentences, total_examples=model.corpus_count, epochs=model.epochs)  # train word vectors


#WORD2VEC TUTORIAL (operations):
vector_comp = model.wv['she']
print("vector: ", vector_comp)

sims = model.wv.most_similar('she', topn=10)  # get other similar words
print(sims)

cosine_sim = model.wv.similarity('her', 'she')
print(cosine_sim)


#https://radimrehurek.com/gensim/models/word2vec.html?fbclid=IwAR0yK8_da0wQdA1jCofwIfOR42Kj4mSVvxTsPG4RS8ckai-s6-YvmyrRFRs#gensim.models.word2vec.Word2Vec

vector:  [-0.01304134  0.01811217  0.0043258   0.00536116 -0.00988783 -0.02774438
  0.0167886   0.05430453 -0.02987418 -0.01788872 -0.00812837 -0.02601906
 -0.02156654  0.01705599  0.00433266 -0.00473014  0.00481395 -0.02250716
 -0.00431839 -0.05969178  0.00393258 -0.00900595  0.02057865 -0.0063015
 -0.00107131 -0.00211714 -0.023861    0.01551093 -0.01824023 -0.00643824
 -0.00055874 -0.00544487  0.00964937 -0.02384784 -0.01500984  0.01381952
  0.01569076 -0.00032666 -0.00539497 -0.02036128  0.01066311 -0.011003
 -0.03353996 -0.00381079  0.00468205 -0.01542626 -0.0086478   0.00259199
  0.02623452  0.02067544  0.01157222 -0.02269591  0.00403526 -0.01339716
  0.00760559  0.01117675  0.01805938 -0.01628358 -0.02156331  0.01580335
 -0.01196434  0.00925338  0.00136045  0.00354243 -0.01364478  0.02254576
  0.00268439  0.00361993 -0.03268065  0.0178864  -0.00033356  0.0170649
  0.01711741  0.00449723  0.02311223  0.00441639 -0.00513552  0.0086604
 -0.00729544 -0.0118693  -0.01909785  0.0092557

## 2 - Create Data sets with wiki and ArXiv from the Pile

## 3 - Debiasing Techniques


### 3.1 - Zhao et al.'s Method Using an Objective Function with Catigorized Words


### 3.2 - Bolukbasi et al.'s Soft-debaising Method

### 3.3 - Savani et al.'s First Order Optimization

## 4 - Measuring Bias

### 4.1 - Direct Bias

### 4.2 - Indirect Bias


For indirect bias, we can define the gender component of words, β(w, v), by defining wg , the contribution from gender, and w⊥ = w −wg, where the word vectors w are unit normal. The equation for β(w, v) from Bolukbasi et al.  is shown below. 
				(w, v)= (wv - wv||w||2||v||2)/(wv)
Larger β(w, v) suggest similarity of embeddings that have no relation (softball and receptionist) which can be largely explained by gender biases in the embedding. Furthermore, to visualize these similarities, we can use k-means to split the embeddings into clusters and analyze the indirect bias that way, as was done in Gohen et al. 

wg = (w · g)g


In [ ]:
def calculate_indirect_metric(g, w, v):
    wg= np.dot(np.dot(w,g),g)
    vg = np.dot(np.dot(v,g),g)
    w_norm_vec = w-wg
    v_norm_vec =v-vg
    w_norm = np.linalg.norm(w_norm_vec)
    v_norm = np.linalg.norm(v_norm_vec)
    
    return (np.dot(w,v) - np.dot(w_norm_vec,v_norm_vec)/(w_norm*v_norm))/np.dot(w,v)


#need to define w, v words which we want to calulate this metric on (in a loop)
    

### 4.3 - WEAT Metric

## 5 - Comparison and Conclusions